In [3]:
import flask
import difflib
import pandas as pd
import re
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from flask import Flask, redirect,url_for, request, render_template
from flask_ngrok import run_with_ngrok
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import cv2
import os
from tensorflow.keras.models import Model
from keras.models import load_model
from tensorflow.keras.models import model_from_json
import numpy as np
from pygame import mixer
import time
from keras import backend as K

In [4]:
books = pd.read_csv(r'\archive\books.csv')

In [5]:
book_tags = pd.read_csv(r'\archive\book_tags.csv')

In [6]:
ratings = pd.read_csv(r'\archive\ratings.csv')

In [7]:
tags = pd.read_csv(r'\archive\tags.csv')

In [8]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(books['authors'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
titles = books['title']
image_url=books['image_url']
indices = pd.Series(books.index, index=books['title'])

def authors_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    book_indices = [i[0] for i in sim_scores]
    data=pd.DataFrame(books['title'].iloc[book_indices])
    data['date']=books['authors']
    return data

In [9]:
tags_join_DF = pd.merge(book_tags, tags, left_on='tag_id', right_on='tag_id', how='inner')
books_with_tags = pd.merge(books, tags_join_DF, left_on='book_id', right_on='goodreads_book_id', how='inner')
tf1 = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix1 = tf1.fit_transform(books_with_tags['tag_name'].head(10000))
cosine_sim1 = linear_kernel(tfidf_matrix1, tfidf_matrix1)
titles1 = books['title']
indices1 = pd.Series(books.index, index=books['title'])

def tags_recommendations(title):
    idx = indices1[title]
    sim_scores = list(enumerate(cosine_sim1[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    book_indices = [i[0] for i in sim_scores]
    data2=pd.DataFrame(books['title'].iloc[book_indices])
    data2['date']=books['authors']
    return data2

In [10]:
def outputsumm(input_text):
    stopWords = set(stopwords.words("english"))
    words = word_tokenize(input_text)
    f = dict()
    for i in words:
        i = i.lower()
        if i in stopWords:
            continue
        if i in f:
            f[i] += 1
        else:
            f[i] = 1
    sentences = sent_tokenize(input_text)
    fsent = dict()
    for j in sentences:
        for x, y in f.items():
            if x in j.lower():
                if j in fsent:
                    fsent[j] += y
                else:
                    fsent[j] = y
    count = 0
    for k in fsent:
        count += fsent[k]
    average = int(count / len(fsent))
    output_text = ''
    for p in sentences:
        if (p in fsent) and (fsent[p] > (1.2 * average)):
            output_text += " "+p
    return output_text

In [11]:
def drowsinessdetector():
    K.clear_session()
    mixer.init()
    sound = mixer.Sound(r'\Drowsiness detection\Drowsiness detection\alarm.wav')

    face = cv2.CascadeClassifier(r'\Drowsiness detection\Drowsiness detection\haar cascade files\haarcascade_frontalface_alt.xml')
    leye = cv2.CascadeClassifier(r'\Drowsiness detection\Drowsiness detection\haar cascade files\haarcascade_lefteye_2splits.xml')
    reye = cv2.CascadeClassifier(r'\Drowsiness detection\Drowsiness detection\haar cascade files\haarcascade_righteye_2splits.xml')
    lbl=['Close','Open']

    model = load_model(r'\Drowsiness detection\Drowsiness detection\models\cnncat2.h5')
    path = os.getcwd()
    cap = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_COMPLEX_SMALL
    count=0
    score=0
    thicc=2
    rpred=[99]
    lpred=[99]
    while (cap.isOpened()):
        ret, frame = cap.read()
        height,width = frame.shape[:2] 

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
        left_eye = leye.detectMultiScale(gray)
        right_eye =  reye.detectMultiScale(gray)

        cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )
        for (x,y,w,h) in faces:
            cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )

        for (x,y,w,h) in right_eye:
            r_eye=frame[y:y+h,x:x+w]
            count=count+1
            r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
            r_eye = cv2.resize(r_eye,(24,24))
            r_eye= r_eye/255
            r_eye=  r_eye.reshape(24,24,-1)
            r_eye = np.expand_dims(r_eye,axis=0)
            rpred = model.predict_classes(r_eye)
            if(rpred[0]==1):
                lbl='Open' 
            if(rpred[0]==0):
                lbl='Closed'
                break

        for (x,y,w,h) in left_eye:
            l_eye=frame[y:y+h,x:x+w]
            count=count+1
            l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)  
            l_eye = cv2.resize(l_eye,(24,24))
            l_eye= l_eye/255
            l_eye=l_eye.reshape(24,24,-1)
            l_eye = np.expand_dims(l_eye,axis=0)
            lpred = model.predict_classes(l_eye)
            if(lpred[0]==1):
                lbl='Open'   
            if(lpred[0]==0):
                lbl='Closed'
                break

        if(rpred[0]==0 and lpred[0]==0):
            score=score+1
            cv2.putText(frame,"Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
            # if(rpred[0]==1 or lpred[0]==1):
        else:
            score=score-1
            cv2.putText(frame,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)

        if(score<0):
            score=0   
        cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
        if(score>15):
            #person is feeling sleepy so we beep the alarm
            cv2.imwrite(os.path.join(path,'image.jpg'),frame)
            try:
                sound.play()
            except:  # isplaying = False
                pass
            if(thicc<16):
                thicc= thicc+2
            else:
                thicc=thicc-2
            if(thicc<2):
                    thicc=2
            cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc) 
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    K.clear_session()

In [12]:
app = flask.Flask(__name__, template_folder=r'\template', static_folder=r'\static')
run_with_ngrok(app)

@app.route('/',methods=['GET','POST'])
def home():
    return render_template('home.html') 

@app.route('/byauthor',methods=['GET','POST'])
def byauthor():
    if flask.request.method == 'GET':
        return(flask.render_template('byauthor.html'))
            
    if flask.request.method == 'POST':
        m_name = flask.request.form['movie_name']  #movie=book
        m_name = m_name.title()
        try: 
            result_final = authors_recommendations(m_name)
            names = []
            dates = []
            for i in range(len(result_final)):
                names.append(result_final.iloc[i][0])
                dates.append(result_final.iloc[i][1])

            return flask.render_template('i3.html',movie_names=names,movie_date=dates,search_name=m_name)
        except KeyError:
            
            return(flask.render_template('i2.html',name=m_name))

@app.route('/bytag',methods=['POST','GET'])
def bytag():
    if flask.request.method=='GET':
        return(flask.render_template('bytag.html'))
    
    if flask.request.method=='POST':
        m_name=flask.request.form['movie_name']
        m_name=m_name.title()
        try:
            result_final=tags_recommendations(m_name)
            names=[]
            dates=[]
            for i in range(len(result_final)):
                names.append(result_final.iloc[i][0])
                dates.append(result_final.iloc[i][1])
            return flask.render_template('i3.html',movie_names=names,movie_date=dates,search_name=m_name)
        except KeyError:
            return(flask.render_template('i2.html',name=m_name))
        
@app.route('/gettext',methods=['POST','GET'])
def gettext():
    try:
        return render_template('gettext.html')
    except ZeroDivisionError:
        return render_template('gettext.html')
    return render_template('gettext.html') 

@app.route('/textsummary',methods=['POST','GET'])
def textsummary():
    try:
         if request.method == 'POST':
            textvalue = request.form.get("textarea", None)
            return render_template('textsummary.html', res=outputsumm(textvalue))
    except ZeroDivisionError:
        return render_template('gettext.html')
    return render_template('textsummary.html')
    
@app.route('/drowsiness',methods=['POST','GET'])
def drowsiness():
    try:
        drowsinessdetector()
    except AttributeError:
        return render_template('home.html') 
    return render_template('home.html') 

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Dec/2020 14:21:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2020 14:21:24] "GET /favicon.ico HTTP/1.1" 404 -
Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\Ishan\anaconda3\envs\tf-gpu\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\Ishan\anaconda3\envs\tf-gpu\lib\threading.py", line 1177, in run
    self.function(*self.args, **self.kwargs)
  File "C:\Users\Ishan\anaconda3\envs\tf-gpu\lib\site-packages\flask_ngrok.py", line 70, in start_ngrok
    ngrok_address = _run_ngrok()
  File "C:\Users\Ishan\anaconda3\envs\tf-gpu\lib\site-packages\flask_ngrok.py", line 38, in _run_ngrok
    tunnel_url = j['tunnels'][0]['public_url']  # Do the parsing of the get
IndexError: list index out of range



In [13]:
import tensorflow as tf; print(tf.__version__) #originally install 2.1.0

1.14.0


In [14]:
#pip install tensorflow==1.14

In [15]:
#pip install keras==2.2.5